<a href="https://colab.research.google.com/github/rutuja-patil24/CMPE-258-Deep_Learning/blob/main/Assignment_05/02_Intermediate_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install --quiet tensorflow datasets transformers matplotlib seaborn

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00


In [2]:
# Enable GPU acceleration
tf.config.optimizer.set_jit(True)

In [3]:
# Load AG News dataset
dataset = load_dataset("ag_news")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [4]:
# Convert dataset to Pandas DataFrame
df = pd.DataFrame(dataset['train'])
df.columns = ['text', 'label']
df['label'] = df['label'].astype(int)

In [5]:
# Reduce dataset size (Train on 10% of data)
df = df.sample(frac=0.1, random_state=42)  # Take only 10% of the data

# Map labels to class names
label_map = {0: "World", 1: "Sports", 2: "Business", 3: "Science/Technology"}
df['category'] = df['label'].map(label_map)


In [6]:
# Split dataset into training & validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].values, df['label'].values, test_size=0.2, stratify=df['label'], random_state=42
)

In [7]:
# ✅ Load Hugging Face Transformer Model & Tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
# ✅ Tokenize text
def tokenize_data(texts):
    return tokenizer(list(texts), padding=True, truncation=True, return_tensors="tf")

train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)

In [9]:
# ✅ Convert dataset into TensorFlow format
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings), train_labels
)).shuffle(len(train_texts)).batch(32).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings), val_labels
)).batch(32).prefetch(tf.data.AUTOTUNE)

In [10]:
# ✅ Load Transformer Model for Sequence Classification (TF Version)
bert_model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4, from_pt=False)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [11]:
# ✅ Fix: Use `tf.keras.optimizers.AdamW`
optimizer = tf.keras.optimizers.AdamW(learning_rate=2e-5, weight_decay=0.01)

# ✅ Fix: Use `from_logits=True` in the loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [12]:
# ✅ Compile the Model
bert_model.compile(optimizer=optimizer,
                   loss=loss_fn,
                   metrics=["accuracy"])

# ✅ Train the Model Using Hugging Face Dataset Pipeline
history = bert_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1  # Reduce epochs for quick training
)

300/300 [==============================] - 402s 1s/step - loss: 0.4390 - accuracy: 0.8761 - val_loss: 0.2657 - val_accuracy: 0.9112


In [13]:
# ✅ Evaluate Model
test_loss, test_acc = bert_model.evaluate(val_dataset)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")


75/75 [==============================] - 23s 302ms/step - loss: 0.2657 - accuracy: 0.9112

✅ Test Accuracy: 0.9112


In [14]:
# ✅ Function to Predict and Visualize
def predict_news(text_samples):
    encodings = tokenize_data(text_samples)
    predictions = bert_model.predict(dict(encodings)).logits
    predicted_labels = np.argmax(predictions, axis=1)
    for text, label in zip(text_samples, predicted_labels):
        print(f"📰 News: {text[:100]}... → **Predicted Category: {label_map[label]}**")

In [15]:
# ✅ Example Predictions
sample_news = [
    "NASA successfully launches new space station mission.",
    "The stock market crashes amid economic concerns.",
    "Manchester United wins the Champions League final!",
    "New advancements in artificial intelligence are reshaping technology."
]
predict_news(sample_news)


1/1 [==============================] - 3s 3s/step
📰 News: NASA successfully launches new space station mission.... → **Predicted Category: Science/Technology**
📰 News: The stock market crashes amid economic concerns.... → **Predicted Category: Business**
📰 News: Manchester United wins the Champions League final!... → **Predicted Category: Sports**
📰 News: New advancements in artificial intelligence are reshaping technology.... → **Predicted Category: Science/Technology**
